## データの取得・確認

In [1]:
import sys
sys.path.append(r'C:/Users/koki5/Dropbox/Jupyter/ScratchML')

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import model_selection
from termcolor import cprint

from common.module.evaluation import score


iris = datasets.load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_target = pd.Series(data=iris.target, name='target')

X_train, X_test, y_train, y_test = model_selection.train_test_split(iris_data, iris_target)

cprint('Iris Data (samples: {}, features: {})'.format(*iris_data.shape), 'blue', attrs=['bold'])
cprint('Iris Target (samples: {})'.format(*iris_target.shape), 'blue', attrs=['bold'])

Iris Data (samples: 150, features: 4)
Iris Target (samples: 150)


In [2]:
iris_data.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [3]:
iris_target.unique()

array([0, 1, 2], dtype=int64)

|  データセット  |  説明  |
| ---- | ---- |
|  sepal length (cm)  |  がく片の長さ  |
|  sepal width (cm)  |  がく片の幅  |
|  petal length (cm)  |  花弁の長さ  |
|  petal width (cm)  |  花弁の幅  |
|  target  |  目的変数／(setosa: 0, versicolor: 1, virginica: 2)  |

## データの可視化

In [4]:
import seaborn as sns
df_concat = pd.concat([iris_data, iris_target], axis=1)
sns.pairplot(df_concat, hue='target')

## データ前処理

In [5]:
# One Hot encoding
y_train_encoded = pd.get_dummies(y_train)
y_train_encoded.head(3)

,0,1,2
25,1,0,0
38,1,0,0
64,0,1,0


In [6]:
def standardize(X):
    """標準化する.
    
    X: データセット
    """
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)


def add_ones(X):
    """特徴量の0列目に1を加える.
    
    X: データセット
    """
    X['0'] = 1
    return X.sort_index(axis=1, ascending=True)


X_train_std = standardize(X_train)
X_test_std = standardize(X_test)

# バイアス項を追加
X_train_std = add_ones(X_train_std)
X_test_std = add_ones(X_test_std)

X_train_std.head(3)

,0,petal length (cm),petal width (cm),sepal length (cm),sepal width (cm)
25,1,-1.208893,-1.290815,-1.024090,-0.108345
38,1,-1.380136,-1.290815,-1.756985,-0.108345
64,1,-0.067274,0.140584,-0.291195,-0.333059


## モデルの構築・訓練

In [7]:
class LogisticRegression(object):
    """勾配降下法に基づくロジスティック回帰."""
    
    
    def __init__(self, alpha=0.01, eps=1e-6):
        """
        
        ---パラメータ-------
        alpha: 学習率
        n_iter: 収束の目安
        --------------------
        
        ---属性-------------
        _w: パラメータ（重み）
        --------------------

        """
        self.alpha = alpha
        self.eps = eps
        
        self._w = 0
        
        
    def fit(self, X, y, C=100):
        """訓練データで学習する.
        
        ---パラメータ-------
        X: 訓練データセット
        y: ターゲット
        --------------------
        
        """
        # パラメータの初期値はランダムに設定する
        theta = np.random.rand(y.shape[1], X.shape[1])
        error = self.J(X, y, theta)
        diff = np.array([1, 1, 1])
        count = 0
        
        # コスト関数の値が殆ど変動しなくなったら収束とする
        while diff.sum() > self.eps:
            # パラメータの更新処理
            grad = (1 / len(y)) * (np.dot((self.h(X, theta) - y).T, X))
            theta = theta - self.alpha * grad
            
            # 収束状況を確認するための処理
            current_error = self.J(X, y, theta)
            diff = error - current_error
            error = current_error
            count += 1
            print('\r【{}回目】Error: {}, Diff: {}, Grad: {}'.format(count, error.sum(), diff.sum(), grad.sum()), end='')
            
        self._w = theta
        return self
    
    
    def predict(self, X):
        """クラスラベルを予測する.
        
        X: テストデータセット
        """
        return self.h(X, self._w).argmax(1)

            
    def activate(self, z):
        """ソフトマックス関数（活性化関数）.
        
        z: 仮説関数の予測値
        """
        return np.exp(z) / np.sum(np.exp(z), axis=1).reshape([z.shape[0], 1])
    

    def h(self, X, theta):
        """仮説関数.
        
        X: 訓練データセット
        theta: パラメータ（重み）
        """
        return self.activate(np.dot(X, theta.T))
    

    def J(self, X, y, theta, C=100):
        """目的関数.
        
        X: 訓練データセット
        y: ターゲット
        theta: パラメータ（重み）
        """
        delta = 1e-7 # np.log()に0が混在しないようにする
        return - (1 / len(y)) * (np.sum(y * np.log(self.h(X, theta) + delta) + (1 - y) * np.log(1 - self.h(X, theta) + delta)))

In [8]:
lr = LogisticRegression()
lr.fit(X_train_std, y_train_encoded)

【45953回目】Error: 0.1430767204074133, Diff: 9.999819611875016e-07, Grad: -1.1817803680091998e-1718

## 予測・評価

In [9]:
predict = pd.DataFrame(lr.predict(X_train_std), columns=['target'])
predict.head(10)

,target
0,0
1,0
2,1
3,2
4,2
5,1
6,2
7,1
8,0
9,0


In [10]:
train_score = score(predict, y_train)
test_score = score(pd.DataFrame(lr.predict(X_test_std), columns=['target']), y_test)
cprint('train_score: {}\ntest_score: {}'.format(train_score, test_score), 'red', attrs=['bold'])

train_score: 0.9732142857142857
test_score: 0.9736842105263158
